
# 정규식 생성기 Regular Expression Generator 

Haskell로 정규식을 생성하는 프로그램을 3단계에 걸쳐 작성해 나가는 예제를 소개한다.
2018년 2학기 관련 과목 강의들의 초반부에 이 예제를 시작으로 하여 함수형 프로그래밍과 정규식을 동시에 학습해서 시간을 절약하는 것이 목표이다.

## 정규식 문법을 하스켈 데이타 타입으로 정의하기

정규식의 문법은 다음과 같이 정의된다.
$$
\begin{array}{l}
c \in \Sigma \\
R ::= \varnothing \mid \varepsilon \mid c \mid R \cdot R \mid R + R \mid R{*}
\end{array}
$$

우선 언어를 구성하는 가장 기본적 단위인 글자의 범위를 정해야 하는데 바로 $\Sigma$가 언어를 구성하는 알파벳의 집합이다. 예를 들면 영어 표기를 구성하는 $\Sigma=\{a,b,c,\ldots,x,y,z\}$라고 대략 비유할 수 있다. 영어는 정규언어도 아니고 더구나 형식언어가 아닌 자연언어이기 때문에 물론 그냥 비유일 뿐이다. 컴퓨터 과학에서 다루는 형식언어 중 구조가 가장 단순한 저급언어인 기계어 등의 이진수로 이루어진 언어들의 경우 $\Sigma=\{0,1\}$이 된다. 위에서는 $\Sigma$의 원소인 글자를 $c$로 표기했다.

(수업시간에 구체적 문법 concrete syntax 추상 문법 또는 요약 문법 abstract syntax 에 대해 설명할 필요가 있다)

정규식 $R$을 구성하는 가장 기초적인 방법은 $\varnothing$, $\varepsilon$, $c$ 이렇게 세 가지이며,
기존의 정규식으로부터 새로운 정규식을 구성하는 방법도 중위(infix) 이항연산자(binary operator)인 $\cdot$과 $+$ 그리고 후위(postfix) 단항연산자(unary operator)인 $*$를 이용하는 세 가지이다.
편의상 $\cdot$은 생략하기도 한다. 즉, $c_1 \cdot c_2$라고 쓰는 대신 $c_1c_2$라고 간단히 쓰기도 한다. 그리고 $\cdot$을 곱셈에 비유할 수 있고 $+$를 덧셈에 비유할 수 있기 때문에 $\cdot$가 $+$보다 우선적으로 결합한다. 예컨대, $(c_1 \cdot c_2)+ c_3$의 경우 괄호를 생략하여 $c_1 \cdot c_2 + c_3$라고 쓰기도 한다. 하지만 $c_1 \cdot (c_2 + c_3)$에서 괄호를 생략한다면 다른 정규식을 나타내게 되어버린다. 세 연산자 중에 가장 결합 우선순위가 높은 것은 $*$이다. 예컨대 $c_1 \cdot (c_2{*})$에서 괄호를 생략하여 $c_1 \cdot c_2{*}$라고 간단히 쓰기도 한다. 참고로 $*$는 0번 이상의 반복을 의미하며 Kleene star 연산자라고 불린다. (수업에서 이거 하기 전에 reflexive transtivie closure 등 relation에 대해서도 다뤄야 좋은데 ...)

정규식을 구성하는 여섯 가지 방법을 하스켈 데이타 타입으로 옮겨서 정의하면 아래와 같다.

In [1]:
data RE -- 정규식 데이타 타입
  = Empty
  | Epsilon       -- ""
  | Alphabet Char -- "a", "b"
  | Concat RE RE
  | Union RE RE
  | Kleene RE
  deriving Show

엄밀히 말하자면 수학적 정의보다는 좀 느슨하게 데이타 타입을 정의했다. 구체적으로 느슨한 부분을 지적하자면 세번째 경우에 해당하는 `Alphabet Char`이다. 수학적 정의를 그대로 따르자면 일단 $\Sigma$에 해당하는 타입을 예컨데 `Sigma`라는 하스켈 데이타 타입을 먼저 정의하거나 `RE`를 타입 인자를 받는 타입 생성자로 만들어 `Sigma` 파라메터로 한다던가 하고 `Alphabet Sigma`라고 하면 수식 표현으로 정의한 것과 더 일치할 것이다.

위 `RE` 데이타타입 정의에서는 편의상 하스켈에서 지원하는 문자를 그대로 그냥 활용하기 위해 하스켈에서 제공하는 문자 타입인 `Char`를 썼다.

즉 $\Sigma$를 따로 정의하거나 타입 파라메터로 일반화하는 대신
$\Sigma$를 하스켈 문자 타입인 `Char`로 고정해 버린 것이다.

프로그래밍 언어에서 값을 분류하는 개념이 타입이다.
그리고 그러한 타입 및 타입 생성자를 분류하는 개념을 kind라고 하며 아래와 같이 `:kind` 명령어로 타입 및 타입 생성자의 kind를 알아볼 수 있다.

In [2]:
:kind RE -- 앞서 정의한 정규식 문법
:kind Int -- 하스켈에서 제공하는 범위가 유한한 정수 타입
:kind [] -- 리스트 타입 생성자 type constuctor
:kind [] Int -- 정수 리스트 타입; 일반적인 전위(prefix) 표기법으로 []를 Int에 적용
:kind [Int] -- 정수 리스트 타입; [] 타입 생성자를 위한 특별한 표기법을 지원한다

RE :: *

Int :: *

[] :: * -> *

[] Int :: *

[Int] :: *

정의된 데이타 생성자의 타입을 알아보려면 다음과 같이 하면 된다.
다른 값이나 식들도 마찬가지로 `:type` 명령어를 통해 알아볼 수 있다.

In [3]:
:type Empty
:type Epsilon
:type Alphabet
:type Concat
:type Union
:type Kleene

Empty :: RE

Epsilon :: RE

Alphabet :: Char -> RE

Concat :: RE -> RE -> RE

Union :: RE -> RE -> RE

Kleene :: RE -> RE

In [3]:
import IHaskell.Display

ppRE r = Display [html(formatRE r)]

formatRE Empty = "∅"
formatRE Epsilon = "ε"
formatRE (Alphabet c) = c:""
formatRE (Concat r1 r2) = formatRE r1 ++ formatRE r2
formatRE (Union r1 r2) = "(" ++ formatRE r1 ++ "+" ++ formatRE r2 ++ ")"
formatRE (Kleene r) = "(" ++ formatRE r ++ ")*"

In [5]:
ppRE(Union (Alphabet '0') 
           (Concat (Alphabet '1')
                   (Kleene (Union (Alphabet '0')
                                  (Alphabet '1')))))

(0+1((0+1))*)

IHaskell 환경에서는 유니코드가 그냥 터미널에서 출력하는 것처럼 되지 않아서
IHaskell에서 지원하는 HTML 출력을 통해 유니코드 글자를 출력한다.

`formatRE`는 하스켈에서 정의한 정규식 타입(RE)의 값을 유니코드 문자를 포함한 수식 표현에 가까운 문자열로 변환하는 함수이다. `ppRE`는 `formatRE`로 변환된 문자열을 IHaskell의 Dispaly 모듈에서 제공하는 HTML 출력 기능을 이용해 노트북에 나타내주기 위한 함수이다.

유니코드가 지원되는 터미널 창에서 코드를 옮겨 프로그래밍하는 경우라면 그냥 변환된 문자열을 `putStr`이나 `putStrLn`같은 표준 출력 함수를 사용하면 된다.

## 정규식의 의미

위에서 정의한 RE는 정규식이라는 표현식의 문법구조이다. 이러한 문법으로 나타나고자 하는 것은 언어 즉 문자열의 집합이다.
알파벳 $\Sigma$로 만들 수 있는 모든 가능한 문자열의 전체 집합이 $\Sigma^{*}$이므로
언어란 $\Sigma^{*}$의 부분집합이다. 따라서 알파벳이 $\Sigma$인 가능한 모든 언어의 집합은 $\Sigma^{*}$의 모든 부분집합의 집합, 즉 $\Sigma^{*}$의 멱집합(powerset)이다. 멱집합 표기는 $2^{\Sigma^{*}}$ 또는 $\wp(\Sigma^{*})$와 같은 두 가지 표기법을 주로 쓴다.
참고로 $*$ 위첨자가 의미하는 바는 어떤 집합을 알파벳으로 삼는 모든 문자열의 집합이다.

정규식이 나타내고자하는 언어가 바로 정규식의 의미라고 할 수 있다. 정규식의 집합을 $\mathcal{R}$이라고 부르자.
$R$의 원소 하나하나, 즉 각각의 정규식을 그 정규식이 언어에 대응시키는 다음과 같은 타입의 함수를 정의한다면
수학적으로 명확하에 정규식 문법에 의미를 부여할 수 있게 된다.

$$\mathcal{L} : \mathcal{R} \to 2^{\Sigma^{*}} \quad\text{또는}\quad \mathcal{L} : \mathcal{R} \to \wp(\Sigma^{*})$$

이렇게 문법(syntax)를 의미(samantic)에 대응시키는 함수를 일반적으로 의미 함수(semantic function)이라고 부른다.
참고로 의미 함수라는 개념은 정규식과 유한 오토마타에서만 아니라 언어를 다루는 영역에서 일반적으로 사용된다.

주어진 정규식이 대표하는 언어를 정의하는 함수 $\mathcal{L}$은 다음과 같이 정의할 수 있다.

$$
\begin{array}{lcl}
\mathcal{L}(\varnothing) &=& \{\} \\
\mathcal{L}(\varepsilon) &=& \{ \varepsilon \} \qquad\text{길이 0인 빈 문자열인 $\varepsilon$ 원소만 있는 집합}\\
\mathcal{L}(c) &=& \{ c \} \qquad\text{문자 c하나로 이루어진 길이 1인 문자열 원소만 있는 집합}\\
\mathcal{L}(R_1 \cdot R_2) &=& \{ uv \mid u\in \mathcal{L}(R_1),~v\in\mathcal{L}(R_2)\} \\
\mathcal{L}(R_1 + R_2) &=& \mathcal{L}(R_1) \cup \mathcal{L}(R_2) \\
\mathcal{L}(R{*}) &=& \mathcal{L}(R)^{*}
\end{array}
$$

지금부터는 정규식의 의미함수 $\mathcal{L}$을 하스켈 프로그램으로 작성해 보겠다. 세 단계에 하스켈로 의미함수를 작성을 시도한다. 하스켈 코드에서는 의미함수 $\mathcal{L}$을 `genRE`, `genRE'`, `genRE''`와 같은 이름의 하스켈 함수로 정의한다. 세 함수들 중에는 두번째가 첫번째보다 좀 낫고 세번째가 두번째보다 좀더 낫긴 하지만 이 세 함수 모두 우리가 보통 원하는 성질의 열거 알고리듬(enumeration algorithm)으로 원소를 나열하는 의미함수는 아니다. 바람직한 열거 알고리듬으로 작성된 정규식의 의미함수를 어떻게 만들어야 할지는 다른 노트북에서 별도로 다루기로 한다.

### 의미함수: 첫번째 시도

의미함수는 정규식으로부터 언어, 즉 문자열의 집합을 생성하므로 하스켈 프로그래밍에 이미 충분히 익숙하다면 집합을 다루기 위한 데이타 타입을 직접 정의해서 쓰거나 `Data.Set` 집합 라이브러리를 사용할 수도 있다. 여기서는 그냥 간단히 집합을 리스트로 나타나겠다.
즉 의미함수를 정규식으로부터 문자열의 집합을 생성하는 아래와 같은 `RE -> [String]` 타입의 함수로 작성하는 것이다.

In [6]:
genRE :: RE -> [String]
genRE Empty          = []
genRE Epsilon        = [ "" ]
genRE (Alphabet c)   = [ c:"" ]
genRE (Concat r1 r2) = [s1++s2 | s1<-genRE r1, s2<-genRE r2]
genRE (Union r1 r2)  = genRE r1 ++ genRE r2 -- 집합을 리스트로 표현하므로 합집합 연산을 (++)로 대신
genRE (Kleene r)     = genRE (Union Epsilon (Concat r (Kleene r)))

마지막 Kleene star에 경우를 제외하면 다른 부분은 수학적 정의와 가까운 구조를 갖고 있다.

마지막 Kleene star에 대한 genRE의 정의는 아래와 같이 식 전개를 통해 유도된 것이다.

$$\begin{array}{rcl}
\mathcal{L}(R{*})
&=& \mathcal{L}(\epsilon + R + RR + RRR + RRRR + \ldots ) \\
&=& \mathcal{L}(\epsilon + R \cdot(\epsilon + R + RR + RRRR +\ldots)) \\
&=& \mathcal{L}(\epsilon + R \cdot R{*})
\end{array}$$

genRE를 순수하게 수학적인 수식으로 보자면 틀리지 않은 정의다.
하지만 이것을 프로그램으로서 실행했을 때
genRE는 우리가 보통 기대하는 바람직한 방식으로
원소를 빠짐없이 나열하도록 동작하지 못한다.
여러가지 정규식에 대한 아래 실행 예시들을 살펴보자.

In [6]:
-- 문자열을 Concat으로 이어진 정규식으로 변환해주는 유틸리티 함수
string2re :: String -> RE
string2re "" = Epsilon
string2re s  = foldr1 Concat (map Alphabet s)

_1 = Alphabet '1'
_0 = Alphabet '0'
_00 = string2re "00"
_01 = string2re "01"
_10 = string2re "10"
_11 = string2re "11"

string2re "00000"

Concat (Alphabet '0') (Concat (Alphabet '0') (Concat (Alphabet '0') (Concat (Alphabet '0') (Alphabet '0'))))

In [7]:
ppRE Empty
genRE Empty

ppRE Epsilon
genRE Epsilon

ppRE _0
genRE _0

ppRE _1
genRE _1

ppRE (Concat _0 _1)
genRE (Concat _0 _1)

ppRE (Union _0 _1)
genRE (Union _0 _1)

ppRE (Union _00 _11)
genRE (Union _00 _11)

ppRE (Concat (Union _00 _11) (Union _01 _10))
genRE (Concat (Union _00 _11) (Union _01 _10))

ppRE (Kleene _1)
take 10 $ genRE (Kleene _1)

ppRE (Kleene _01)
take 10 $ genRE (Kleene _01)

ppRE (Union (Kleene _0) (Kleene _1))
take 10 $ genRE (Union (Kleene _0) (Kleene _1)) -- 0, 00, 000 같은 것만 나오고 1이 들어가는 건 안나타남

ppRE (Concat (Kleene _0) (Kleene _1))
take 10 $ genRE (Concat (Kleene _0) (Kleene _1)) -- 1, 11, 111 같은 것만 나오고 0이 들어가는 건 안타나남

ppRE (Kleene (Union _00 _11))
take 10 $ genRE (Kleene (Union _00 _11)) -- 00, 0000 같은 것만 나오고 11이 들어가는 건 안나타남

∅

[]

ε

[""]

0

...

["","0","00","000","0000","00000","000000","0000000","00000000","000000000"]

(0)*(1)*

["","1","11","111","1111","11111","111111","1111111","11111111","111111111"]

((00+11))*

["","00","0000","000000","00000000","0000000000","000000000000","00000000000000","0000000000000000","000000000000000000"]

위에서 맨 마지막 세 개를 제외한 다른 예제들은 기대한 대로 작동한다. 하지만 맨 마지막 두 예제는 1이나 11이 들어가는 것은 나오지 않고 0이나 00이 들어간 것만 나열되어 나온다. 

("recursively enummerable"에 대한 개념을 대략적으로나마 강의시간에 소개한다.)

`genRE`에서 `Union`의 경우에 한쪽으로 몰려서 나오지 않도록 아래와 같이 수정한 함수 `genRE'`를 정의할 수 있다.

In [8]:
-- (++) 대신 두 리스트의 원소를 번갈아 가며 나타내는 방식으로 합치는 merge 함수 정의
merge []     ys = ys
merge (x:xs) ys = x : merge ys xs

genRE' Empty          = []
genRE' Epsilon        = [ "" ]
genRE' (Alphabet c)   = [ c:"" ]
genRE' (Concat r1 r2) = [s1++s2 | s1<-genRE' r1, s2<-genRE' r2]
genRE' (Union r1 r2)  = genRE' r1 `merge` genRE' r2 -- 연산자 ++ 대신 `merge` 사용
genRE' (Kleene r)     = genRE' (Union Epsilon (Concat r (Kleene r)))

In [9]:
merge [1,3,5,7,9] [2,4,6,8,10]

[1,2,3,4,5,6,7,8,9,10]

In [10]:
-- 이항 함수를 역따옴표로 감싸면 중위(infix) 연산자로 사용할 수 있다
[1,3,5,7,9] `merge` [2,4,6,8,10]

[1,2,3,4,5,6,7,8,9,10]

In [11]:
-- 중위(infix) 연산자는 괄호로 감싸면 전위(prefix) 연산자, 즉 보통 함수처럼  사용할 수 있다

3 + 4

(+) 3 4

7

7

In [12]:
genRE' Empty
genRE' Epsilon
genRE' _0
genRE' _1
genRE' (Concat _0 _1)
genRE' (Union _0 _1)
genRE' (Union _00 _11)
genRE' (Concat (Union _00 _11) (Union _01 _10))
take 10 $ genRE' (Kleene _1)
take 10 $ genRE' (Kleene _01)

ppRE (Union (Kleene _0) (Kleene _1))
take 10 $ genRE' (Union (Kleene _0) (Kleene _1))

-- 1, 11, 111 같은 것만 나오고 0이 들어가는 건 안타나남
ppRE (Concat (Kleene _0) (Kleene _1))
take 10 $ genRE' (Concat (Kleene _0) (Kleene _1)) 

-- 00, 0000 같은 것만 나오고 11이 들어가는 건 안나타남
ppRE (Kleene (Union _00 _11))
take 10 $ genRE' (Kleene (Union _00 _11)) 

[]

[""]

["0"]

["1"]

["01"]

...

["","","0","1","00","11","000","111","0000","1111"]

(0)*(1)*

["","1","11","111","1111","11111","111111","1111111","11111111","111111111"]

((00+11))*

["","00","0000","000000","00000000","0000000000","000000000000","00000000000000","0000000000000000","000000000000000000"]

하지만 여전히 마지막 두 경우는 의 경우에는 한쪽으로 몰려서 나온다.
이게 왜 그렇게 되는지는 스스로 생각해 보라.

$0{*}1{*}$의 경우를 한쪽으로 쏠리지 않고 고르게 번갈아 나타나도록 하려면 리스트 조건제시법의 형태로 두 문자열을 이어붙인 문자열의 집합을 구하는 방식이 아니라, 이차원에 배열했을 때 모서리에서부터 대각선 방향을 따라가는 방식에 가깝게 `Concat`을 처리하는 방식을 바꾸면 된다. 

In [13]:
diagCartProd (x:xs) (y:ys) = (x,y) : ([(x,y) | x<-xs] `merge` diagCartProd xs ys `merge` [(x,y) | y<-ys])
diagCartProd _ _ = []

diagConcProd (x:xs) (y:ys) = (x++y) : ([x++y | x<-xs] `merge` diagConcProd xs ys `merge` [x++y | y<-ys]) 
diagConcProd _ _ = []

genRE'' Empty          = []
genRE'' Epsilon        = [ "" ]
genRE'' (Alphabet c)   = [ c:"" ]
genRE'' (Concat r1 r2) = genRE'' r1 `diagConcProd` genRE'' r2
genRE'' (Union r1 r2)  = genRE'' r1 `merge` genRE'' r2
genRE'' (Kleene r)     = genRE'' (Union Epsilon (Concat r (Kleene r)))

In [14]:
:type diagCartProd
:type diagConcProd

diagCartProd [0,1,2,3,4,5,6,7,8] [0,1,2,3,4,5,6,7,8]
length $ diagCartProd [0,1,2,3,4,5,6,7,8] [0,1,2,3,4,5,6,7,8]
diagConcProd ["a","aa","aaa","aaaa","aaaaa"] ["b","bb","bbb","bbbb","bbbbb"]

diagCartProd :: forall a t. [a] -> [t] -> [(a, t)]

diagConcProd :: forall a. [[a]] -> [[a]] -> [[a]]

[(0,0),(1,0),(0,1),(1,1),(0,2),(2,0),(0,3),(2,1),(0,4),(3,0),(0,5),(1,2),(0,6),(4,0),(0,7),(2,2),(0,8),(5,0),(1,3),(6,0),(3,1),(7,0),(1,4),(8,0),(3,2),(1,5),(4,1),(1,6),(2,3),(1,7),(5,1),(1,8),(3,3),(6,1),(2,4),(7,1),(4,2),(8,1),(2,5),(4,3),(2,6),(5,2),(2,7),(3,4),(2,8),(6,2),(4,4),(7,2),(3,5),(8,2),(5,3),(3,6),(5,4),(3,7),(6,3),(3,8),(4,5),(7,3),(5,5),(8,3),(4,6),(6,4),(4,7),(6,5),(4,8),(7,4),(5,6),(8,4),(6,6),(5,7),(7,5),(5,8),(7,6),(8,5),(6,7),(7,7),(6,8),(8,6),(8,7),(7,8),(8,8)]

81

["ab","aab","abb","aabb","abbb","aaab","abbbb","aaabb","abbbbb","aaaab","aabbb","aaaaab","aaabbb","aabbbb","aaaabb","aabbbbb","aaaabbb","aaaaabb","aaabbbb","aaaabbbb","aaabbbbb","aaaaabbb","aaaaabbbb","aaaabbbbb","aaaaabbbbb"]

* 생각해 볼 문제: `diagCartProd`나 `diagCongProd`는 안타깝지게도 대각선 비스무레하게 하려고 노력하지만 진짜 대각선을 따라가지는 못하고 있다. 제대로 대각선 순서대로 나오게 하려면 어떻게 바꾸면 될까?

In [15]:
genRE'' Empty
genRE'' Epsilon
genRE'' _0
genRE'' _1
genRE'' (Concat _0 _1)
genRE'' (Union _0 _1)
genRE'' (Union _00 _11)
genRE'' (Concat (Union _00 _11) (Union _01 _10))
take 10 $ genRE'' (Kleene _1)
take 10 $ genRE'' (Kleene _01)
take 10 $ genRE'' (Union (Kleene _0) (Kleene _1))
take 15 $ genRE'' (Concat (Kleene _0) (Kleene _1))

-- 00만 연속되거나 11만 연속되는 것만 나타나고 0011같은 것이 나타나지 않는다
take 35 $ genRE'' (Kleene (Union (Kleene _00) (Kleene _11)))

[]

[""]

["0"]

["1"]

["01"]

...

["","1","11","111","1111","11111","111111","1111111","11111111","111111111"]

["","01","0101","010101","01010101","0101010101","010101010101","01010101010101","0101010101010101","010101010101010101"]

["","","0","1","00","11","000","111","0000","1111"]

["","0","1","01","11","00","111","001","1111","000","11111","011","111111","0000","1111111"]

["","","","","","","00","","00","","11","","","00","0000","","00","00","1111","","","11","000000","","11","","111111","00","","0000","00000000","","11","00","11111111"]

하지만 Kleene star가 여러 번 겹쳐 나타나는 정규식에서는 여전히 우리가 원하는 방식으로 나타나지 않는다. 예컨대, 정규식 $((00){*}+(11){*}){*}$에 대한 의미함수를 생각할 때 우리는 이런 식으로 나타나길 기대한다.

$$\mathcal{L}(((00){*}+(11){*}){*})
= \{\epsilon,00,11,0000,0011,1100,1111,000000,000011,001100,110000,001111,110011,111100,111111,\ldots\}$$ 

왜냐하면 $\mathcal{L}(((00){*}+(11){*}){*}) = \mathcal{L}((00+11){*})$이라는 것을 곰곰히 생각해 보면 알 수 있기 때문이다.

In [16]:
ppRE (Kleene (Union (Kleene _00) (Kleene _11)))
take 40 $ genRE'' (Kleene (Union (Kleene _00) (Kleene _11)))

ppRE (Kleene (Union _00 _11))
take 35 $ genRE'' (Kleene (Union _00 _11))

(((00)*+(11)*))*

["","","","","","","00","","00","","11","","","00","0000","","00","00","1111","","","11","000000","","11","","111111","00","","0000","00000000","","11","00","11111111","00","","1111","0000000000",""]

((00+11))*

["","00","11","0000","1100","0011","1111","000000","110000","001100","111100","000011","110011","001111","111111","00000000","11000000","00110000","11110000","00001100","11001100","00111100","11111100","00000011","11000011","00110011","11110011","00001111","11001111","00111111","11111111","0000000000","1100000000","0011000000","1111000000"]